In [8]:
import torch
import torch.nn as nn

In [9]:
device = torch.device('mps')

In [10]:
class DiscriminatorUCSB(nn.Module):
    def __init__(self):
        super(DiscriminatorUCSB, self).__init__()
        self.ndf = 64  # Size of feature maps in discriminator
        self.nc = 3  # Number of channels of the training images

        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=self.ndf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf, out_channels=self.ndf*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*2, out_channels=self.ndf*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*4, out_channels=self.ndf*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*8, out_channels=self.ndf*16, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ndf * 16),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(in_channels=self.ndf*16, out_channels=1, kernel_size=3, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.network(input)

In [11]:
class GeneratorUCSB(nn.Module):
    def __init__(self):
        super(GeneratorUCSB, self).__init__()

        self.nz = 100  # Size of z latent vector
        self.ngf = 64  # Size of feature maps in generator
        self.nc = 3  # Number of channels in the training images

        self.network = nn.Sequential(
            nn.ConvTranspose2d(in_channels=self.nz, out_channels=self.ngf*16, kernel_size=3, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(self.ngf*16),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*16, out_channels=self.ngf*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*8, out_channels=self.ngf*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*4, out_channels=self.ngf*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf*2, out_channels=self.ngf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=self.ngf, out_channels=self.nc, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.network(input)

In [12]:
discriminator = DiscriminatorUCSB().to(device)
generator = GeneratorUCSB().to(device)

In [13]:
real = torch.zeros(1, 3, 96, 96)
real = real.to(device)

noise = torch.randn(1, 100, 1, 1)
noise = noise.to(device)
fake = generator(noise)
print(fake.size())

torch.Size([1, 3, 96, 96])


In [14]:
output = discriminator(real)
print(output.size())

torch.Size([1, 1, 1, 1])
